In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model_notoverfit.h5'
# dimensions of our images.
img_width, img_height = 150, 150

In [3]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 5
batch_size = 16

In [4]:
model = applications.VGG16(weights='imagenet', include_top=False,input_shape=(150,150,3))

In [5]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [6]:
top_model.load_weights(top_model_weights_path)

In [9]:
from keras.models import Model
model = Model(input= model.input, output= top_model(model.output))

/home/aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  


In [10]:
for layer in model.layers[:25]:
    layer.trainable = False

In [11]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


/home/aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/aditya/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=5, validation_data=<keras_pre..., steps_per_epoch=125, validation_steps=800)`


Epoch 1/5
125/125 [==============================] - 1492s 12s/step - loss: 0.2917 - acc: 0.8730 - val_loss: 0.2892 - val_acc: 0.8962
Epoch 2/5
125/125 [==============================] - 1466s 12s/step - loss: 0.2783 - acc: 0.8845 - val_loss: 0.2892 - val_acc: 0.8962
Epoch 3/5
125/125 [==============================] - 1458s 12s/step - loss: 0.2733 - acc: 0.8875 - val_loss: 0.2892 - val_acc: 0.8962
Epoch 4/5
125/125 [==============================] - 1457s 12s/step - loss: 0.3017 - acc: 0.8755 - val_loss: 0.2892 - val_acc: 0.8962
Epoch 5/5
125/125 [==============================] - 1457s 12s/step - loss: 0.2963 - acc: 0.8810 - val_loss: 0.2892 - val_acc: 0.8962
